# What affects methane chemical lifetime?

- methane
- VOCs
- NOx
- Ozone
- HCs
- N2O
- climate

Here I suggest an override of the alpha scaling factor for methane that is calculated from multiple species.

Ozone itself is a function of other precursors: we do not include ozone as a direct influence on methane lifetime, and restrict ourselves to directly emitted anthropogenic species.

Gill Thornhill published two papers on methane lifetime: one on the chemical adjustments to lifetime, and one on the climate adjustments. Both effects will be included. We will 

1. take AerChemMIP multi-model means
2. run the lifetime relationship to individual AerChemMIP models
3. run a Monte Carlo

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import time
import scipy.stats

from tqdm import tqdm
from fair21 import SpeciesID, Category, Config, Species, RunMode, Scenario, ClimateResponse, RunConfig, FAIR
from fair21.defaults import species_config_from_default

## Temperature data

Use observations 1850-2020, then simulate an SSP3-7.0 climate with a linear warming rate to 4C in 2100.

In [ ]:
df_temp = pd.read_csv('../data/forcing/AR6_GMST.csv')
gmst = np.zeros(751)
gmst[100:271] = df_temp['gmst'].values
gmst[271:351] = np.linspace(gmst[270], 4, 80)
gmst[351:] = 4

## Get emissions and concentrations 

In [ ]:
samples = 500

In [ ]:
df_emis = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
input = {}

In [ ]:
conc_species = ['CH4', 'N2O', 'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
    'HFC-365mfc', 'HFC-4310mee']

for species in conc_species:
    species_rcmip_name = species.replace("-", "")
    input[species] = df_conc.loc[
        (df_conc['Scenario']=='ssp370') & (df_conc['Variable'].str.endswith(species_rcmip_name)) & 
        (df_conc['Region']=='World'), '1750':'2500'
    ].interpolate(axis=1).values.squeeze()

In [ ]:
emis_species = ['CO', 'VOC', 'NOx']
for species in emis_species:
    input[species] = df_emis.loc[
        (df_emis['Scenario']=='ssp370') & (df_emis['Variable'].str.endswith(species)) & 
        (df_emis['Region']=='World'), '1750':'2500'
    ].interpolate(axis=1).values.squeeze()

In [ ]:
input['temp'] = gmst

In [ ]:
# species_ids = {
#     'ch4': SpeciesID('CH4', Category.CH4),
#     'co2': SpeciesID('CO2', Category.CO2, run_mode=RunMode.CONCENTRATION),
#     'n2o': SpeciesID('N2O', Category.N2O, run_mode=RunMode.CONCENTRATION),
#     'co': SpeciesID('CO', Category.SLCF_OZONE_PRECURSOR),
#     'voc': SpeciesID('VOC', Category.SLCF_OZONE_PRECURSOR),
#     'nox': SpeciesID('NOx', Category.SLCF_OZONE_PRECURSOR),
# }

In [ ]:
radiative_efficiency = pd.Series({
    'HFC-125': 0.23378,
    'HFC-134a': 0.16714,
    'HFC-143a': 0.168,
    'HFC-152a': 0.10174,
    'HFC-227ea': 0.27325,
    'HFC-23': 0.19111,
    'HFC-236fa': 0.25069,
    'HFC-245fa': 0.24498,
    'HFC-32': 0.11144,
    'HFC-365mfc': 0.22813,
    'HFC-4310mee': 0.35731
})

In [ ]:
hfc_erf = {}
hfc_sum = 0
for species in ['HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32',
    'HFC-365mfc', 'HFC-4310mee']:
    hfc_erf[species] = (input[species][269] * radiative_efficiency[species]/1000)
    hfc_sum = hfc_sum + hfc_erf[species]

Use 1850 and 2014 emissions or concentrations corresponding to methane lifetime changes in Thornhill et al. 2021.

Could we also take into account the fact that there are multiple loss pathways for CH4:
- tropospheric OH loss is 560 Tg/yr
- chlorine oxidation, 11 Tg/yr, assumed not included in AerChemMIP models
- stratospheric loss is 31 Tg/yr, assumed not included in AerChemMIP models
- soil uptake, 30 Tg/yr, not included in AerChemMIP models

Saunois (2020): 90% of sink is OH chemistry in troposphere and is 553 [476–677] Tg CH4 yr−1, which is close to the IPCC number of 560, (chapter 5)

Chapter 6 only give time constants for soil uptake and the combined chemistry loss (trop OH + chlorine + stratosphere). 

In [ ]:
# def alpha_scaling(
#     input,
#     baseline,
#     normalisation,
#     beta
# ):
#     lifetime_scaling = 1
#     for species in ['CH4', 'N2O', 'CO', 'VOC', 'NOx', 'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea',
#                     'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32', 'HFC-365mfc', 'HFC-4310mee', 'temp']:
#         lifetime_scaling = lifetime_scaling + (
#             (input[species]-baseline[species])/normalisation[species] * beta[species]
#         )
#     return lifetime_scaling

def alpha_scaling_exp(
    input,
    baseline,
    normalisation,
    beta,
    hfc_erf,
    hfc_sum,
):
    log_lifetime_scaling = 0
    for species in ['CH4', 'N2O', 'CO', 'VOC', 'NOx', 'temp']:
        log_lifetime_scaling = log_lifetime_scaling + (
            np.log(1 + (input[species]-baseline[species])/normalisation[species] * beta[species])
        )
    for species in ['HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea',
                    'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32', 'HFC-365mfc', 'HFC-4310mee']:
        log_lifetime_scaling = log_lifetime_scaling + (
            np.log(1 + 
                (
                    (input[species]-baseline[species])/(normalisation[species]) * beta['HC'] * hfc_erf[species]/hfc_sum
                )
            )
        )
    return np.exp(log_lifetime_scaling)

In [ ]:
normalisation = {}
for species in ['CH4', 'N2O', 'CO', 'VOC', 'NOx', 'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea',
                    'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32', 'HFC-365mfc', 'HFC-4310mee']:
    normalisation[species] = input[species][264] - input[species][100]
normalisation['temp'] = 1

In [ ]:
baseline = {}
for species in ['CH4', 'N2O', 'CO', 'VOC', 'NOx', 'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea',
                    'HFC-23', 'HFC-236fa', 'HFC-245fa', 'HFC-32', 'HFC-365mfc', 'HFC-4310mee']:
    baseline[species] = input[species][100]
baseline['temp'] = 0

In [ ]:
parameters = {}

parameters['AerChemMIP_mean'] = {
    'base': 10.0,
    'CH4': +0.22,
    'NOx': -0.33,
    'CO': 0,
    'VOC': +0.19,
    'HC': -0.0037,
    'N2O': -0.02,
    'temp': 0.0012,
}

parameters['UKESM'] = {
    'base': 8,
    'CH4': +0.22,
    'NOx': -0.25,
    'CO': 0,
    'VOC': +0.11,
    'HC': -0.0049,
    'N2O': -0.0012,
    'temp': 0.0043
}

In [ ]:
lifetime_scaling = {}

In [ ]:
for setup in ['AerChemMIP_mean', 'UKESM']:
    lifetime_scaling[setup] = alpha_scaling_exp(
        input,
        baseline,
        normalisation,
        parameters[setup],
        hfc_erf,
        hfc_sum,
    )

In [ ]:
#pl.plot(np.arange(1750, 2501), aerchemmip_mean[:] * 8.25)
pl.plot(np.arange(1750, 2501), lifetime_scaling['AerChemMIP_mean'] * parameters['AerChemMIP_mean']['base'])
pl.plot(np.arange(1750, 2501), lifetime_scaling['UKESM'] * parameters['UKESM']['base'])

In [ ]:
1/(1/135 + 1/9.7)

In [ ]:
1/(1/120 + 1/200 + 1/150 + 1/11.2)

In [ ]:
# put this into a simple one box model
def one_box(
    emissions,
    gas_boxes_old,
    airborne_emissions_old,
    burden_per_emission,
    lifetime,
    alpha_lifetime,
    partition_fraction,
    pre_industrial_concentration,
    soil_lifetime=135,
    timestep=1,
    natural_emissions_adjustment=0,
):
    
    effective_lifetime = 1/(1/(alpha_lifetime * lifetime) + 1/soil_lifetime)
    decay_rate = timestep/(effective_lifetime)
    decay_factor = np.exp(-decay_rate)
    gas_boxes_new = (
        partition_fraction *
        (emissions-natural_emissions_adjustment) *
        1 / decay_rate *
        (1 - decay_factor) * timestep + gas_boxes_old * decay_factor
    )
    airborne_emissions_new = gas_boxes_new
    concentration_out = (
        pre_industrial_concentration +
        burden_per_emission * (
            airborne_emissions_new + airborne_emissions_old
        ) / 2
    )
    return concentration_out, gas_boxes_new, airborne_emissions_new

In [ ]:
emis_ch4 = df_emis.loc[
    (df_emis['Scenario']=='ssp370') & (df_emis['Variable'].str.endswith('CH4')) & 
    (df_emis['Region']=='World'), '1750':'2500'
].interpolate(axis=1).values.squeeze()

In [ ]:
burden_per_emission = 1 / (5.1352e18 / 1e18 * 16.043 / 28.97)
partition_fraction = 1
pre_industrial_concentration = 729.2
natural_emissions_adjustment = emis_ch4[0]

In [ ]:
conc_ch4 = {}

In [ ]:
for setup in ['UKESM', 'AerChemMIP_mean']:
    conc_ch4[setup] = np.zeros(751)
    gas_boxes = 0
    airborne_emissions = 0
    for i in range(751):
        conc_ch4[setup][i], gas_boxes, airborne_emissions = one_box(
            emis_ch4[i],
            gas_boxes,
            airborne_emissions,
            burden_per_emission,
            parameters[setup]['base'],
            lifetime_scaling[setup][i],
            partition_fraction,
            pre_industrial_concentration,
            soil_lifetime=135,
            timestep=1,
            natural_emissions_adjustment=natural_emissions_adjustment,
        )

In [ ]:
pl.plot(np.arange(1750, 2501), conc_ch4['AerChemMIP_mean'])
pl.plot(np.arange(1750, 2501), conc_ch4['UKESM'])
pl.plot(np.arange(1750, 2501), input['CH4'], color='k')

In [ ]:
pl.plot(np.arange(1750, 2021), conc_ch4['AerChemMIP_mean'][:271])
pl.plot(np.arange(1750, 2021), conc_ch4['UKESM'][:271])
pl.plot(np.arange(1750, 2021), input['CH4'][:271], color='k')

In [ ]:
np.exp(-0.33)

In [ ]:
np.log(1-0.33)